In [92]:
import math
import copy
import pymorphy2
import pandas_profiling
import warnings
import matplotlib.pyplot as plt
import regex as re
import numpy as np
import pandas as pd

from tempfile import mkdtemp
from shutil import rmtree
from time import time
from sklearn import preprocessing
from scipy import optimize
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, ShuffleSplit
from sklearn.naive_bayes import BernoulliNB, ComplementNB, MultinomialNB
from catboost import CatBoostClassifier, cv, Pool, CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
from nltk.stem.snowball import RussianStemmer

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [93]:
%load_ext ipython_anybar
%ipython_anybar_connect 192.168.0.102 1738

The ipython_anybar extension is already loaded. To reload it, use:
  %reload_ext ipython_anybar
Connect to 192.168.0.102:1738


In [94]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           u"\U0001f926-\U0001f937"
                           u"\u200d"
                           u"\u2640-\u2642" 
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [95]:
def load_data(posts_file_name='user_posts.csv', profiles_file_name='user_profiles.csv', samples=1.):
    """
    param samples: can be absolute int number or float for partition.
    
    returns posts, profiles
    """
    profiles = pd.read_csv(profiles_file_name)
    posts = pd.read_csv(posts_file_name)
    
    profiles = profiles.sample(n=samples if type(samples) is int else None,
                              frac=samples if type(samples) is float else None)
    posts = posts[posts.id.isin(profiles.id)]
    
    return posts, profiles

In [96]:
%%time
posts, profiles = load_data(samples=1.)

CPU times: user 9.43 s, sys: 2.48 s, total: 11.9 s
Wall time: 12.6 s


In [97]:
# Lets look reports for our data.
pandas_profiling.ProfileReport(profiles)

Number of variables,28
Number of observations,13543
Total Missing (%),51.4%
Total size in memory,2.6 MiB
Average record size in memory,203.0 B
Numeric,3
Categorical,19
Boolean,2
Date,0
Text (Unique),1
Rejected,3


In [98]:
pandas_profiling.ProfileReport(posts)

Number of variables,2
Number of observations,810929
Total Missing (%),0.0%
Total size in memory,18.6 MiB
Average record size in memory,24.0 B
Numeric,1
Categorical,1
Boolean,0
Date,0
Text (Unique),0
Rejected,0


In [99]:
def combine_data(posts, profiles, profile_columns):
    # This method concatenates posts for each user and adds fields from profiles.
    # Result df index is user id.
    def concatenator(texts):
        result = ' '.join(texts.astype(str))
        return result
    grouped_posts = posts.groupby(by='id', as_index=False).agg({'text': concatenator})
    grouped_posts = grouped_posts.set_index('id')
    profiles_fields = profiles.set_index('id').loc[:,profile_columns]
    
    result = pd.merge(profiles_fields, grouped_posts, on='id')
    return result

In [100]:
target_column_name = 'sex'
combined_data = combine_data(posts, profiles, [target_column_name])
combined_data

,sex,text
id,,
121514,m,Принес вам на ночь историю про одного неудачни...
884718,f,"Фотографии наших учеников, с нашего последнего..."
52990,m,Империя Спасибо Украинскй зомбиленд: взгляд из...
629076,f,ВНИМАНИЕ ВСЕМ ПОСТАМ!<br>Как и обещали - трети...
1806708,f,Внимание! Максимальный репост! <br>Вчера 18 но...
7628805,m,Саксофон от которого бегут мурашки по коже.. Э...
3450353,f,Stream of silens.... События - ужасные трусы: ...
1142304,m,"Смотри, что нашлось на AliExpress Какой ужас....."
1217673,f,"""Ted Toyler"" , 1920-е годы Это кто тут у нас т..."


In [101]:
# Clean data.

# Remove html tags.
combined_data.text.replace('<[^<]+?>', '', regex=True, inplace=True)

# Remove tags started from '#'.
combined_data.text.replace('#[\S]+', '', regex=True, inplace=True)

# Remove contacts links: '[id38042050|Анастасия Темрязанская]'.
combined_data.text.replace('\[[^<]+?\]', '', regex=True, inplace=True)

# Remove emoji.
combined_data.text = combined_data.text.apply(remove_emoji)

combined_data

,sex,text
id,,
121514,m,Принес вам на ночь историю про одного неудачни...
884718,f,"Фотографии наших учеников, с нашего последнего..."
52990,m,Империя Спасибо Украинскй зомбиленд: взгляд из...
629076,f,ВНИМАНИЕ ВСЕМ ПОСТАМ!Как и обещали - третий тр...
1806708,f,Внимание! Максимальный репост! Вчера 18 ноября...
7628805,m,Саксофон от которого бегут мурашки по коже.. Э...
3450353,f,Stream of silens.... События - ужасные трусы: ...
1142304,m,"Смотри, что нашлось на AliExpress Какой ужас....."
1217673,f,"""Ted Toyler"" , 1920-е годы Это кто тут у нас т..."


In [102]:
# Normalization.
# Here we will write 2 methods for tokenization->remove stop words/punctuations/emoji->stem/lem->join to string.
# Using these methods we make 2 new columns for further processing.

def tokenize(text):
    words = word_tokenize(text)
    return words

def remove_stopwords(words):
    stop_words = stopwords.words('russian')
    cleared_words = [word for word in words if word not in stop_words and len(word) > 2]
    return cleared_words

russian_stemmer = RussianStemmer()
def word_stem(word):
    stem = russian_stemmer.stem(word)
    return stem

pymorph = pymorphy2.MorphAnalyzer()
def word_lemma(word):
    lemma = pymorph.parse(word)[0]
    return lemma.normal_form

def normalize(text, norm_func):
    words = tokenize(text)
    without_stopwords = remove_stopwords(words)
    stems = [norm_func(word) for word in without_stopwords]
    joined = ' '.join(stems)
    return joined

def normalize_by_stems(text):
    result = normalize(text, word_stem)
    return result
    
def normalize_by_lemms(text):
    result = normalize(text, word_lemma)
    return result

In [103]:
combined_data['text_stems'] = combined_data.text.apply(normalize_by_stems)
combined_data['text_lemms'] = combined_data.text.apply(normalize_by_lemms)

In [104]:
combined_data

,sex,text,text_stems,text_lemms
id,,,,
121514,m,Принес вам на ночь историю про одного неудачни...,принес ноч истор одн неудачник телефон виде от...,принести ночь история один неудачник телефон в...
884718,f,"Фотографии наших учеников, с нашего последнего...",фотограф наш ученик наш последн занят базов мо...,фотография наш ученик наш последний занятие ба...
52990,m,Империя Спасибо Украинскй зомбиленд: взгляд из...,импер спасиб украинск зомбиленд взгляд изнутр ...,империя спасибо украинскть зомбиленд взгляд из...
629076,f,ВНИМАНИЕ ВСЕМ ПОСТАМ!Как и обещали - третий тр...,вниман всем пост как обеща трет трек грядущ ал...,внимание весь пост как обещать три трек грядущ...
1806708,f,Внимание! Максимальный репост! Вчера 18 ноября...,вниман максимальн репост вчер ноябр стадион пе...,внимание максимальный репост вчера ноябрь стад...
7628805,m,Саксофон от которого бегут мурашки по коже.. Э...,саксофон котор бегут мурашк коже.. эт музык ле...,саксофон который бежать мурашка коже.. это муз...
3450353,f,Stream of silens.... События - ужасные трусы: ...,Stre silens ... событ ужасн трус случа одн сби...,stream silens ... событие ужасный трус случать...
1142304,m,"Смотри, что нашлось на AliExpress Какой ужас.....",смотр нашл AliExpress как ужас ... оригина htt...,смотреть найтись aliexpress какой ужас ... ори...
1217673,f,"""Ted Toyler"" , 1920-е годы Это кто тут у нас т...",Ted Toyler 1920-е год эт жалк лисенок STEIFF п...,ted toyler 1920-е год это жалкий лисёнок steif...


In [105]:
"""
Next we will compare 5 models:
1. stems + CountVectorizer + MultinomialNB
2. stems + TfidfVectorizer + MultinomialNB
3. lemmas + CountVectorizer + MultinomialNB
4. lemmas + TfidfVectorizer + MultinomialNB

All models we will implement using sklearn. 
We will search best model params on partial dataset and then make cross validation score on full dataset.
"""

'\nNext we will compare 5 models:\n1. stems + CountVectorizer + MultinomialNB\n2. stems + TfidfVectorizer + MultinomialNB\n3. lemmas + CountVectorizer + MultinomialNB\n4. lemmas + TfidfVectorizer + MultinomialNB\n\nAll models we will implement using sklearn. \nWe will search best model params on partial dataset and then make cross validation score on full dataset.\n'

In [106]:
# Prepare datasets for params searching and testing.
df_train, df_test = train_test_split(combined_data, test_size=0.5)
_, df_train_for_params_searching = train_test_split(df_train, test_size=0.01)

In [107]:
def find_best_model(df, column_name, vectorizer):
    cachedir = mkdtemp()
    pipeline = Pipeline([
        vectorizer,
        ('cls', MultinomialNB())
    ], memory=cachedir)

    parameters = {
        'vect__min_df': [1, 5, 0.1],
        'vect__max_df': [0.5, 0.75, 1.],
        'vect__ngram_range': [(1, 1), (1, 2)],
    }
    
    grid_search = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1)
    t0 = time()    
    grid_search.fit(df.loc[:, column_name], df['sex'])
    print("done in %0.3fs" % (time() - t0))
    rmtree(cachedir)
    
    return grid_search.best_estimator_

In [108]:
# Search best params for models.
column_names = ['text_stems', 'text_lemms']
vectorizers = [('CountVectorizer', ('vect', CountVectorizer())), ('TfidfVectorizer', ('vect', TfidfVectorizer()))]
best_models = {}
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    for column_name in column_names:
        for vectorizer_title, vectorizer in vectorizers:
            best_model = find_best_model(df_train_for_params_searching, column_name, vectorizer)
            best_models[(column_name, vectorizer_title)] = best_model

done in 164.079s
done in 154.279s
done in 150.999s
done in 157.552s


In [109]:
# Compare models.
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    t0 = time()    
    for ((column_name, vectorizer_title), model) in best_models.items():
        model.fit(df_train[column_name], df_train['sex'])
        y_pred = model.predict(df_test[column_name])
        y_test = df_test['sex']
        score = accuracy_score(y_pred, y_test)
        print(f'{column_name} {vectorizer_title}: {score}')
    
    print("done in %0.3fs" % (time() - t0))

Pipeline(memory='/var/folders/9_/w6j66k193r3d0n6vsctvbjj00000gn/T/tmp6624eu34',
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.5, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)), ('cls', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

text_stems CountVectorizer: 0.7790084388185654


Pipeline(memory='/var/folders/9_/w6j66k193r3d0n6vsctvbjj00000gn/T/tmpew190b_7',
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.5, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...rue,
        vocabulary=None)), ('cls', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

text_stems TfidfVectorizer: 0.5896624472573839


Pipeline(memory='/var/folders/9_/w6j66k193r3d0n6vsctvbjj00000gn/T/tmpvxrjhy1h',
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.75, max_features=None, min_df=5,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)), ('cls', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

text_lemms CountVectorizer: 0.7533403656821378


Pipeline(memory='/var/folders/9_/w6j66k193r3d0n6vsctvbjj00000gn/T/tmpctq4evz6',
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.5, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...rue,
        vocabulary=None)), ('cls', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

text_lemms TfidfVectorizer: 0.5896624472573839
done in 434.946s


In [ ]:
# From above scores best model for user sex prediction is applying CountVectorizer on stems.